In [2]:
# Translated to .py by Meritxell Pacheco
# 2017
# Adapted to PandasBiogeme by Michel Bierlaire
# Sun Oct 21 23:15:31 2018

import pandas as pd
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, DefineVariable
from biogeme.models import loglogit

pandas = pd.read_table("lpmc02.dat")
database = db.Database("LPMC",pandas)
pd.options.display.float_format = '{:.3g}'.format

globals().update(database.variables)

# Exclude
# exclude = (  ArrivalTimeHours_1   ==  -1  )
# database.remove(exclude)
  
# Choice
chosenAlternative = travel_mode

#Parameters to be estimated+ (  BestAlternative_4   *  4  )
# Arguments:
#   1  Name for report. Typically, the same as the variable
#   2  Starting value
#   3  Lower bound
#   4  Upper bound
#   5  0: estimate the parameter, 1: keep it fixed
Constant1	 = Beta('Constant1',0,None,None,1)
Constant2	 = Beta('Constant2',0,None,None,0)
Constant3	 = Beta('Constant3',0,None,None,0)
Constant4	 = Beta('Constant4',0,None,None,0)
Cost	 = Beta('Cost',0,None,None,0)
Total_TT	 = Beta('Total_TT',0,None,None,0)

# Define here arithmetic expressions for name that are not directly
# available from the data

dur_public = DefineVariable('dur_public', dur_pt_access + dur_pt_rail + dur_pt_bus + dur_pt_int ,database)
cost_public = DefineVariable('cost_public', cost_transit ,database)
cost_driving = DefineVariable('cost_driving', cost_driving_fuel + cost_driving_ccharge ,database)


# Utilities

#Opt1 = walking
#Opt2 = cycling
#Opt3 = public transport
#Opt4 = driving

Opt1 = Constant1 + Total_TT * dur_walking
Opt2 = Constant2 + Total_TT * dur_cycling
Opt3 = Constant3 + Cost * cost_public + Total_TT * dur_public
Opt4 = Constant4 + Cost * cost_driving + Total_TT * dur_driving
V = {1: Opt1,2: Opt2,3: Opt3,4: Opt4}
av = {1: 1,2: 1,3: 1,4: 1}

# The choice model is a logit, with availability conditions
logprob = loglogit(V,av,chosenAlternative)
biogeme  = bio.BIOGEME(database,logprob)
biogeme.modelName = "logit_lpmc_model0"
results = biogeme.estimate()
# Get the results in a pandas table
pandasResults = results.getEstimatedParameters()
print(pandasResults)
print(f"Nbr of observations: {database.getNumberOfObservations()}")
print(f"LL(0) =    {results.data.initLogLike:.3f}")
print(f"LL(beta) = {results.data.logLike:.3f}")
print(f"rho bar square = {results.data.rhoBarSquare:.3g}")
print(f"Output file: {results.data.htmlFileName}")

           Value  Std err  t-test  p-value  Rob. Std err  Rob. t-test  \
Constant2  -3.87    0.103   -37.4        0         0.109        -35.3   
Constant3 -0.511    0.053   -9.65        0        0.0546        -9.36   
Constant4  -1.27   0.0738   -17.2        0        0.0825        -15.4   
Cost      -0.174   0.0136   -12.8        0         0.013        -13.4   
Total_TT   -5.47    0.183   -29.8        0         0.212        -25.8   

           Rob. p-value  
Constant2             0  
Constant3             0  
Constant4             0  
Cost                  0  
Total_TT              0  
Nbr of observations: 5000
LL(0) =    -6931.472
LL(beta) = -4572.870
rho bar square = 0.34
Output file: logit_lpmc_model0.html
